<a href="https://colab.research.google.com/github/GerardoFesta/3DFER_SE4AI/blob/main/2DCnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow

TRAIN_PATH = "train"
TEST_PATH ="test"

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Questo commento serve per provare il funzionamento dei commit con colab

# **Dataset Loading**

Unzip del dataset (Temporaneamente da caricare da file)

In [ ]:
import zipfile



zip_ref = zipfile.ZipFile('fer2013.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall() #Extracts the files into the /tmp folder
zip_ref.close()

Associazione path delle immagini - classe delle immagini

In [19]:
import os
class_dir=os.listdir(TRAIN_PATH+'/')
folder_names = class_dir
label_dict = {folder_names[i]:i for i in range(len(folder_names))}
num_classes=len(label_dict)
train_image_filenames = []
train_labels = []

# iterate through each folder and collect filenames and labels
for folder_name in folder_names:
    folder_path = os.path.join(TRAIN_PATH, folder_name)
    for filename in os.listdir(folder_path):
        train_image_filenames.append(os.path.join(TRAIN_PATH+"/"+folder_name, filename))
        train_labels.append(label_dict[folder_name])

# create pandas dataframe
train_df = pd.DataFrame({'filename': train_image_filenames, 'emotion': train_labels})

print(len(train_df))


test_image_filenames = []
test_labels = []
class_dir=os.listdir(TEST_PATH+'/')
for folder_name in folder_names:
    folder_path = os.path.join(TEST_PATH, folder_name)
    for filename in os.listdir(folder_path):
        test_image_filenames.append(os.path.join(TEST_PATH+"/"+folder_name, filename))
        test_labels.append(label_dict[folder_name])

test_df = pd.DataFrame({'filename': test_image_filenames, 'emotion': test_labels})

28709


Caricamento matrici delle immagini

In [20]:
train_df['img_as_matrix'] = train_df['filename'].apply(lambda path: cv2.imread(path,cv2.IMREAD_GRAYSCALE))
test_df['img_as_matrix'] = test_df['filename'].apply(lambda path: cv2.imread(path,cv2.IMREAD_GRAYSCALE))


Creazione del dataset, estendendo la classe dataset di PyTorch

In [21]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image = self.data.iloc[index]['img_as_matrix'] 
        label = self.data.iloc[index]['emotion'] 

        # Esegui le trasformazioni se definite
        if self.transform is not None:
            image = self.transform(image)

        return image, label

Creazione dei Loader

In [22]:
train_df.drop(columns=["filename"], inplace=True)
test_df.drop(columns=["filename"], inplace=True)
train_df.at[0,"img_as_matrix"]
train_dataset = CustomDataset(train_df, transform=transforms.ToTensor())
test_dataset = CustomDataset(test_df, transform=transforms.ToTensor())

batch_size = 32  

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# **Rete convolutiva**

In [23]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        
        self.fc1 = nn.Linear(64 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = self.relu(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        
        x = self.fc3(x)
        
        return x
